In [21]:
import pickle
import numpy as np

coordinates = np.asarray([tuple(map(int, x.decode('UTF-8').split('-'))) for x in pickle.load( open( "bootstrap/1e4_bootstrap/0.pkl", "rb" ))], dtype=int)
ind = np.lexsort((coordinates[:, 1], coordinates[:, 0]))
print(coordinates.shape, coordinates)
print(ind.shape)
print(coordinates[ind, :])
print(np.max(coordinates[:, 0]))

(10000, 2) [[        2  74254374]
 [        5 100848386]
 [        4  69033052]
 ...
 [        1 195245236]
 [        6  24087753]
 [        3  44935109]]
(10000,)
[[        1    720468]
 [        1    728242]
 [        1    814813]
 ...
 [        9 141051632]
 [        9 141087372]
 [        9 141109979]]
9


In [27]:
import gzip
with gzip.open('../split_gen_miss/chr.20.gen.variants.txt.gz', 'rt') as f:
    chrom_coord = []
    for line in f:
        chrom, pos, _, ref, alt = line.strip().split('\t', maxsplit=5)[:5]
        is_biallelic_snp = 1 if len(ref) == 1 and len(alt) == 1 and ref != '.' and alt != '.' else 0
        chrom_coord.append((int(chrom), int(pos), is_biallelic_snp))
        
chrom_coord = np.asarray(chrom_coord, dtype=int)
np.save('chr.20.gen.coordinates', chrom_coord)
print(chrom_coord.shape, chrom_coord)

(1707845, 3) [[      20    60097        1]
 [      20    60175        1]
 [      20    60309        1]
 ...
 [      20 62965480        1]
 [      20 62965513        1]
 [      20 62965516        1]]
